## AutoEncoder Wind Farm B

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GaussianNoise
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
import os

# Custom EarlyStopping callback monitoring L2 norm reconstruction error on validation set
class L2NormEarlyStopping(tf.keras.callbacks.Callback):
    def __init__(self, validation_data, patience=3):
        super().__init__()
        self.X_val = validation_data[0]
        self.y_val = validation_data[1]  # same as X_val for AE
        self.patience = patience
        self.best_weights = None
        self.best_l2_norm = np.Inf
        self.wait = 0
        self.stopped_epoch = 0

    def on_epoch_end(self, epoch, logs=None):
        val_pred = self.model.predict(self.X_val, verbose=0)
        l2_norm_error = np.linalg.norm(self.X_val - val_pred)
        print(f"Epoch {epoch+1}: Validation L2 norm reconstruction error = {l2_norm_error:.6f}")

        if l2_norm_error < self.best_l2_norm:
            self.best_l2_norm = l2_norm_error
            self.wait = 0
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print(f"Early stopping triggered at epoch {self.stopped_epoch + 1}. Restoring best model weights.")
                self.model.set_weights(self.best_weights)

# Dataset list for WindFarm B 
dataset_ids = [2, 7, 19, 21, 23, 27, 34, 52, 53, 74, 77, 82, 83, 86, 87]

def calculate_relative_wind_direction(awd, nd):
    return (awd - nd + 540) % 360 - 180


save_dir = r"D:\Master Thesis New Data Set\AutoEncoder\Wind Farm B\Power_Final"
os.makedirs(save_dir, exist_ok=True)

for dataset_id in dataset_ids:
    print(f"\n=== Processing WindFarm A Dataset {dataset_id} ===")

    df = pd.read_csv(
        rf"D:\Master Thesis New Data Set\Final DataSet\Wind Farm B Ultimate\{dataset_id}_WindFarm_B.csv",
        delimiter=','
    )

    df['time_stamp'] = pd.to_datetime(df['time_stamp'])
    df['train_test'] = df['train_test'].astype(str).str.strip().str.lower()

    # Ensure positive power values
        # Set negative power to 0
    df['power_62_avg'] = df['power_62_avg'].apply(lambda x: max(x, 0))

        # Calculate relative wind direction
    df['relative_wind_direction'] = calculate_relative_wind_direction(df['sensor_4_avg'], df['sensor_21_avg'])

        # Step 1: Filter for normal operation
    df_filtered = df[
        (df['sensor_10_avg'] <= 2) & 
        (df['sensor_10_avg'] >= -5) & 
        (df['relative_wind_direction'] >= -10) & 
        (df['relative_wind_direction'] <= 10) & 
        (df['status_type_id'] == 0) &
        (df['train_test'] == 'train')
     ].dropna(subset=['wind_speed_61_avg', 'power_62_avg'])

    df_sorted = df_filtered.sort_values(by='wind_speed_61_avg')

        # Normalize power
    original_max_power = df_sorted['power_62_avg'].max()
    rated_power = original_max_power  # normalized
    power_threshold = 0.04 * rated_power
    print(f"Rated power: {rated_power:.4f}, Using power threshold: {power_threshold:.4f} (4% of rated power)")

    # Enhanced plausibility filter (Fraunhofer + your power threshold)
    normal_df = df[
        (df['status_type_id'] == 0) &
        (df['train_test'] == 'train') &
        ~((df['wind_speed_61_avg'] >= 3) & 
          (df['wind_speed_61_avg'] <= 25) & 
          (df['power_62_avg'] <= power_threshold))  # Ignore low power within normal wind speed range
    ].copy()

    exclude_cols = ['time_stamp', 'train_test', 'id', 'asset_id']
    feature_cols = [c for c in df.columns if c not in exclude_cols]

    X = normal_df[feature_cols].apply(pd.to_numeric, errors='coerce').fillna(0).values

    if X.shape[0] < 50:
        print(f"⚠️ Skipping dataset {dataset_id} due to insufficient training data after filtering.")
        continue

    X_train, X_val = train_test_split(X, test_size=0.25, random_state=42)

    input_dim = X_train.shape[1]

    # Updated AE architecture, learning rate, and no GaussianNoise
    autoencoder = Sequential([
        Dense(40, activation='relu', input_shape=(input_dim,)),
        Dense(15, activation='relu'),
        Dense(40, activation='relu'),
        Dense(input_dim, activation='linear')
                ])
    autoencoder.compile(optimizer=Adam(learning_rate=0.003), loss='mse')


    l2_early_stop = L2NormEarlyStopping(validation_data=(X_val, X_val), patience=3)

    autoencoder.fit(
        X_train, X_train,
        validation_data=(X_val, X_val),
        epochs=200,
        batch_size=64,
        callbacks=[l2_early_stop],
        verbose=1
    )

    # Validation reconstruction errors (L2 norm per sample)
    reconstructions_val = autoencoder.predict(X_val)
    reconstruction_errors_val = np.linalg.norm(X_val - reconstructions_val, axis=1)

    # NN regressor threshold model
    # Updated threshold NN
    nn_regressor = MLPRegressor(hidden_layer_sizes=(36,), activation='relu', max_iter=300, random_state=42)
    nn_regressor.fit(X_val, reconstruction_errors_val)

    predict_df = df[df['train_test'] == 'prediction'].copy()
    if predict_df.empty:
        print(f"⚠️ No prediction data for dataset {dataset_id}, skipping prediction.")
        continue

    X_predict = predict_df[feature_cols].apply(pd.to_numeric, errors='coerce').fillna(0).values

    reconstructions_pred = autoencoder.predict(X_predict)
    actual_errors = np.linalg.norm(X_predict - reconstructions_pred, axis=1)
    expected_errors = nn_regressor.predict(X_predict)

    gamma = 0.234  # Fraunhofer gamma for Wind Farm B
    is_anomaly = (actual_errors > expected_errors + gamma).astype(int)

    predict_df['predicted_status_type_id'] = np.where(is_anomaly == 1, 1, 0)

    save_path = os.path.join(save_dir, f"{dataset_id}_WindFarmB_status_predictions.csv")
    predict_df.to_csv(save_path, index=False)

    print(f"✅ Saved WindFarm B prediction for Dataset {dataset_id} — Anomalies detected: {sum(is_anomaly)}")

print("\n🎉 All WindFarm B datasets processed. AE-NBM predictions complete.")



=== Processing WindFarm A Dataset 2 ===
Rated power: 1.0290, Using power threshold: 0.0412 (10% of rated power)
Epoch 1/200


C:\Users\49157\miniconda3\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


285/335 ━━━━━━━━━━━━━━━━━━━━ 0s 908us/step - loss: 7888225.0000Epoch 1: Validation L2 norm reconstruction error = 472186.680438
335/335 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 7049853.0000 - val_loss: 122355.7266
Epoch 2/200
317/335 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 83892.0781Epoch 2: Validation L2 norm reconstruction error = 181303.710802
335/335 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 81985.1484 - val_loss: 18038.9102
Epoch 3/200
318/335 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 15989.5664Epoch 3: Validation L2 norm reconstruction error = 165517.996955
335/335 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 15993.6426 - val_loss: 15034.4424
Epoch 4/200
324/335 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 15401.7832Epoch 4: Validation L2 norm reconstruction error = 166894.740117
335/335 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 15383.3115 - val_loss: 15285.5830
Epoch 5/200
289/335 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13961.6504Epoch 5: Validation L2 norm reconstruction error = 156440.

C:\Users\49157\miniconda3\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


328/345 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7713189.5000Epoch 1: Validation L2 norm reconstruction error = 418890.290350
345/345 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 7425255.5000 - val_loss: 93710.1953
Epoch 2/200
335/345 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 42618.3555Epoch 2: Validation L2 norm reconstruction error = 183876.486042
345/345 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 42153.4570 - val_loss: 18056.7090
Epoch 3/200
341/345 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 17241.8281Epoch 3: Validation L2 norm reconstruction error = 179883.613004
345/345 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 17247.9941 - val_loss: 17281.0254
Epoch 4/200
345/345 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 17659.7246Epoch 4: Validation L2 norm reconstruction error = 178737.780743
345/345 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 17658.7891 - val_loss: 17061.5723
Epoch 5/200
334/345 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 17356.8691Epoch 5: Validation L2 norm reconstruction error = 171102.693

...

299/310 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 12040354.0000Epoch 1: Validation L2 norm reconstruction error = 548323.275802
310/310 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 11721335.0000 - val_loss: 178320.1250
Epoch 2/200
303/310 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 94376.2031Epoch 2: Validation L2 norm reconstruction error = 223810.658178
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 93602.5781 - val_loss: 29709.0254
Epoch 3/200
292/310 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 26829.0859Epoch 3: Validation L2 norm reconstruction error = 240365.022216
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 26830.0645 - val_loss: 34266.4766
Epoch 4/200
288/310 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 29432.6348Epoch 4: Validation L2 norm reconstruction error = 182636.693139
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 29576.0117 - val_loss: 19783.4961
Epoch 5/200
301/310 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 20998.4941Epoch 5: Validation L2 norm reconstruction error = 178168.

C:\Users\49157\miniconda3\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


291/313 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9404091.0000Epoch 1: Validation L2 norm reconstruction error = 323631.650650
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 8918567.0000 - val_loss: 61681.1992
Epoch 2/200
294/313 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 25953.9570Epoch 2: Validation L2 norm reconstruction error = 182001.374528
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 25579.8242 - val_loss: 19507.4316
Epoch 3/200
296/313 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 15875.3809Epoch 3: Validation L2 norm reconstruction error = 160538.157899
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 15864.3740 - val_loss: 15177.7461
Epoch 4/200
299/313 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 16382.7432Epoch 4: Validation L2 norm reconstruction error = 165898.743241
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 16333.3896 - val_loss: 16208.2832
Epoch 5/200
282/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 15956.1221Epoch 5: Validation L2 norm reconstruction error = 160718.712

C:\Users\49157\miniconda3\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


350/356 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6652822.0000Epoch 1: Validation L2 norm reconstruction error = 258726.080104
356/356 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 6557598.5000 - val_loss: 34631.4805
Epoch 2/200
348/356 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 24625.4180Epoch 2: Validation L2 norm reconstruction error = 190935.989805
356/356 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 24538.0586 - val_loss: 18861.0664
Epoch 3/200
344/356 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 17620.5527Epoch 3: Validation L2 norm reconstruction error = 188127.305005
356/356 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 17635.5605 - val_loss: 18310.2480
Epoch 4/200
351/356 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 17508.4375Epoch 4: Validation L2 norm reconstruction error = 190344.587344
356/356 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 17512.3711 - val_loss: 18744.4102
Epoch 5/200
342/356 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 17807.6270Epoch 5: Validation L2 norm reconstruction error = 189334.067